<a href="https://colab.research.google.com/github/Amisha1019/assignment_3/blob/main/Tas_2_Null_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install opencv-python tensorflow keras pandas


In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.1 MB/s eta 0:00:00


In [3]:
import streamlit as st
from PIL import Image
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.src.legacy.saving import legacy_h5_format
from keras.models import load_model

In [6]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime

In [7]:
face_model = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


In [28]:
AGE_BUCKETS = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
GENDER_LIST = ['Male', 'Female']


In [12]:
csv_path = "senior_visitors_sample.csv"

In [29]:
df = pd.read_csv(csv_path)


In [30]:
cap = cv2.VideoCapture(0)
person_id = df["Person_ID"].max() if not df.empty else 0

print("Press 'q' to quit the video window")


Press 'q' to quit the video window


In [31]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_model.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), (78.426, 87.769, 114.896), swapRB=False)

        # Predict Gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = GENDER_LIST[gender_preds[0].argmax()]

        # Predict Age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age_bucket = AGE_BUCKETS[age_preds[0].argmax()]
        age_est = int(age_bucket.strip('()').split('-')[1])
        senior = "Yes" if age_est > 60 else "No"

        # Draw info on frame
        label = f"{gender}, {age_bucket}, Senior: {senior}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # Log new record
        person_id += 1
        visit_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_data = pd.DataFrame([[person_id, age_est, gender, senior, visit_time]],
                                columns=["Person_ID", "Age", "Gender", "SeniorCitizen", "Time_of_Visit"])
        new_data.to_csv(csv_path, mode='a', header=False, index=False)

    cv2.imshow("Senior Citizen Identification", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()